In [73]:
import openpyxl   
import pandas as pd
import numpy as np
from pandas import read_excel
import matplotlib.pyplot as plt
import scipy.optimize as scp

In [102]:
raw_data=pd.read_excel("air1.xlsx", engine = 'openpyxl') 
data = pd.DataFrame(raw_data)
data

,Unnamed: 0,Sample,T,ch1,ch2,mean,inverted
0,0,-331,-0.000007,0.000801,0.000297,0.000164,-0.000164
1,1,-330,-0.000007,0.000801,0.000595,0.000193,-0.000193
2,2,-329,-0.000007,0.000801,0.000297,0.000221,-0.000221
3,3,-328,-0.000007,0.000801,0.000000,0.000220,-0.000220
4,4,-327,-0.000007,0.000801,0.000099,0.000245,-0.000245
...,...,...,...,...,...,...,...
131067,131067,130736,0.002615,0.000801,0.000099,0.000225,-0.000225
131068,131068,130737,0.002615,0.000601,0.000099,0.000222,-0.000222
131069,131069,130738,0.002615,0.001002,0.000595,0.000216,-0.000216
131070,131070,130739,0.002615,0.001002,0.000198,0.000217,-0.000217


In [103]:
time_column = data.columns[2]
ch1_column = data.columns[3]
data_column = data.columns[-1]
print(time_column,ch1_column,data_column)
%matplotlib widget
plt.plot(data[time_column],data[ch1_column])

T ch1 inverted


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
def f(x,a,b):
    return a*np.exp(-x/b)
    

In [81]:
points_candidates = np.argwhere(data[ch1_column].to_numpy()[2:]>0.010)

i = 0
points = []
while i<(len(points_candidates)-1):
    distance = points_candidates[i+1][0]-points_candidates[i][0]
    if distance < 200:
        if data[ch1_column].to_numpy()[2:][points_candidates[i][0]]>data[ch1_column].to_numpy()[2:][points_candidates[i+1][0]]:
            points.append(points_candidates[i][0])
        else:
            points.append(points_candidates[i+1][0])
        i = i + 1

    else:
        points.append(points_candidates[i][0])

    i = i + 1

points.append(len(data[ch1_column].to_numpy()[2:])-1)
print(points)

"""
%matplotlib widget
plt.plot(data[time_column].to_numpy()[2:],data[data_column].to_numpy()[2:])
for i in range(len(points)):
    plt.axvline(x= data[time_column].to_numpy()[2:][points[i]], color = 'r')

plt.show()
"""



[1, 2, 5, 7, 8, 10, 13, 15, 16, 19, 21, 22, 24, 27, 29, 31, 32, 34, 36, 39, 41, 42, 45, 46, 48, 51, 53, 54, 56, 59, 61, 62, 64, 67, 68, 70, 73, 75, 76, 78, 80, 83, 85, 87, 89, 90, 92, 95, 96, 98, 101, 102, 105, 107, 108, 110, 113, 115, 116, 119, 120, 123, 124, 127, 128, 130, 133, 135, 136, 138, 141, 143, 145, 146, 149, 150, 153, 155, 156, 158, 161, 163, 165, 166, 168, 171, 173, 174, 177, 179, 180, 182, 185, 186, 188, 191, 193, 194, 197, 198, 200, 202, 205, 207, 208, 210, 213, 214, 216, 219, 220, 222, 225, 227, 228, 231, 233, 234, 236, 239, 240, 242, 244, 247, 248, 251, 253, 254, 257, 259, 260, 262, 265, 266, 268, 271, 273, 274, 277, 279, 280, 282, 285, 287, 288, 291, 293, 294, 296, 298, 300, 303, 305, 306, 309, 310, 313, 314, 316, 319, 320, 322, 325, 326, 328, 331, 333, 334, 336, 339, 340, 342, 345, 347, 348, 351, 353, 354, 357, 359, 361, 362, 365, 366, 369, 370, 373, 374, 377, 379, 380, 382, 385, 387, 388, 391, 392, 394, 397, 399, 400, 402, 405, 407, 408, 410, 413, 414, 416, 419, 420,

"\n%matplotlib widget\nplt.plot(data[time_column].to_numpy()[2:],data[data_column].to_numpy()[2:])\nfor i in range(len(points)):\n    plt.axvline(x= data[time_column].to_numpy()[2:][points[i]], color = 'r')\n\nplt.show()\n"

In [80]:

thaus = []

for i in range(len(points)-1):

    indexes = np.arange(points[i],points[i+1])
    val_between_imp = data[data_column].to_numpy()[2:][indexes]
    peak = np.argmax(val_between_imp)

    times_fit = np.array(data[time_column ].to_numpy()[2:][indexes][peak:] - data[time_column ].to_numpy()[2:][indexes][peak:][0],dtype=np.float32)
    val_fit = np.array((data[data_column].to_numpy()[2:][indexes][peak:]-np.average(data[data_column].to_numpy()[2:][indexes][peak:][-80:])),dtype=np.float32)
    a_init = val_fit[0]
    b_init = -1/(np.log(val_fit[80]/a_init)/times_fit[80])

    params, pcov = scp.curve_fit(f, times_fit, val_fit, [a_init,b_init])
    err = np.sqrt(np.diag(pcov))

    thaus.append(params[1])
    
    """
    %matplotlib widget
    plt.plot(times_fit,val_fit)
    plt.plot(times_fit, f(times_fit, params[0], params[1]), color = 'r')
    """
print(np.average(thaus),np.std(thaus))






IndexError: index 80 is out of bounds for axis 0 with size 1

In [ ]:
2.362955057402244e-06 5.691787963020979e-08
1.2406205788082178e-06 7.728304207621119e-08